In [1]:
import requests
from bs4 import BeautifulSoup
import csv

In [19]:
url = "https://books.toscrape.com/"
response = requests.get(url)
response

<Response [200]>

In [20]:
soup = BeautifulSoup(response.text, 'html.parser')
Data = soup.find_all('article', class_='product_pod')
data_list = []
for Datas in Data:
    title = Datas.h3.a['title']
    price = Datas.find('p', class_='price_color').text
    data_list.append([title,price])
data_list


[['A Light in the Attic', 'Â£51.77'],
 ['Tipping the Velvet', 'Â£53.74'],
 ['Soumission', 'Â£50.10'],
 ['Sharp Objects', 'Â£47.82'],
 ['Sapiens: A Brief History of Humankind', 'Â£54.23'],
 ['The Requiem Red', 'Â£22.65'],
 ['The Dirty Little Secrets of Getting Your Dream Job', 'Â£33.34'],
 ['The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
  'Â£17.93'],
 ['The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
  'Â£22.60'],
 ['The Black Maria', 'Â£52.15'],
 ['Starving Hearts (Triangular Trade Trilogy, #1)', 'Â£13.99'],
 ["Shakespeare's Sonnets", 'Â£20.66'],
 ['Set Me Free', 'Â£17.46'],
 ["Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)", 'Â£52.29'],
 ['Rip it Up and Start Again', 'Â£35.02'],
 ['Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',
  'Â£57.25'],
 ['Olio', 'Â£23.88'],
 ['Mesaerion: The Best Science Fiction Stories 1800-1849', 'Â£37.59'],
 ['Libertarian